In [27]:
%load_ext sql
%sql mysql://iasamori@localhost/?charset=utf8mb4
%sql SHOW DATABASES;


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

9 rows affected.

Database
campione
dla_tutorial
dlatk_lexica
iasamori
iasamori_album
iasamori_test
iasamori_world
information_schema
sql_intro


In [28]:
%config SqlMagic.displaylimit = 200

In [29]:
%sql USE iasamori;

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

++
||
++
++

### Remove non English text and clean URLs and @mentions

In [12]:
!dlatkInterface.py --corpdb iasamori --corptable cmt7 \
    --correl_field message_id --language_filter en --clean_messages




-----
DLATK Interface Initiated: 2025-06-06 02:49:59
-----
SQL QUERY: DROP TABLE IF EXISTS cmt7_en
SQL QUERY: CREATE TABLE cmt7_en LIKE cmt7
SQL QUERY: ALTER TABLE cmt7_en CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
Reading 938,852 messages
   10000 rows written ( 1.390 % done)
   20000 rows written ( 2.775 % done)
   30000 rows written ( 4.128 % done)
   40000 rows written ( 5.496 % done)
   50000 rows written ( 6.849 % done)
   60000 rows written ( 8.189 % done)
   70000 rows written ( 9.542 % done)
   80000 rows written (10.868 % done)
   90000 rows written (12.174 % done)
  100000 rows written (13.470 % done)
  110000 rows written (14.800 % done)
  120000 rows written (16.125 % done)
  130000 rows written (17.470 % done)
  140000 rows written (18.849 % done)
  150000 rows written (20.157 % done)
  160000 rows written (21.489 % done)
  170000 rows written (22.814 % done)
  180000 rows written (24.161 % done)
  190000 rows written (25.489 % done)
  200000 rows written 

### Remove duplicated texts

In [13]:
%%sql
    
ALTER TABLE cmt7_en
ADD COLUMN user_id INT DEFAULT 1;

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

746615 rows affected.

++
||
++
++

In [14]:
!dlatkInterface.py --corpdb iasamori --corptable cmt7_en \
    --correl_field user_id --deduplicate --clean_messages




-----
DLATK Interface Initiated: 2025-06-06 03:04:26
-----
SQL QUERY: DROP TABLE IF EXISTS cmt7_en_dedup
SQL QUERY: CREATE TABLE cmt7_en_dedup LIKE cmt7_en
SQL QUERY: ALTER TABLE cmt7_en_dedup CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
deduplicating messages for 1 'user_id's
-------
Settings:

Database - iasamori
Corpus - cmt7_en
Group ID - user_id
-------
Interface Runtime: 3115.75 seconds
DLATK exits with success! A good day indeed  ¯\_(ツ)_/¯.


### Check tables

In [15]:
a = %sql SHOW TABLES
print(a)

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

5 rows affected.

+------------------------------------------+
|            Tables_in_iasamori            |
+------------------------------------------+
|                   cmt7                   |
|                 cmt7_en                  |
|              cmt7_en_dedup               |
|   feat$1gram$cmt7_en_dedup$message_id    |
| feat$meta_1gram$cmt7_en_dedup$message_id |
+------------------------------------------+


### Make 1gram tokens of comments messages

In [16]:
!dlatkInterface.py --corpdb iasamori --corptable cmt7_en_dedup --correl_field message_id --add_ngrams -n 1




-----
DLATK Interface Initiated: 2025-06-06 03:56:24
-----
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_en_dedup' AND column_name='message_id'
SQL QUERY: DROP TABLE IF EXISTS feat$1gram$cmt7_en_dedup$message_id
SQL QUERY: CREATE TABLE feat$1gram$cmt7_en_dedup$message_id ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id double, feat VARCHAR(36) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin, value INTEGER, group_norm DOUBLE, KEY `correl_field` (`group_id`), KEY `feature` (`feat`)) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_en_dedup' AND column_name='message_id'
SQL QUERY: DROP TABLE IF EXISTS feat$meta_1gram$cmt7_en_dedup$message_id
SQL QUERY: CREATE TABLE feat$meta_1gram$cmt7_en_dedup$message_id ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id double, feat VARCHA

In [17]:
a = %sql SHOW TABLES
print(a)

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

5 rows affected.

+------------------------------------------+
|            Tables_in_iasamori            |
+------------------------------------------+
|                   cmt7                   |
|                 cmt7_en                  |
|              cmt7_en_dedup               |
|   feat$1gram$cmt7_en_dedup$message_id    |
| feat$meta_1gram$cmt7_en_dedup$message_id |
+------------------------------------------+


### Select out clean comments from comments_7 table. This table will be called cmt7_cln

In [19]:
%load_ext rpy2.ipython

In [20]:
%%R

source("./psych290_data/helper_files/psych290RcodeV1.R")

require(tidyverse)
require(RSQLite)
require(ggthemes)
require(ggpubr)
require(grid)
require(reshape2)
require(psych)
require(lm.beta)
library(caret)

library(DBI)
library(RMySQL)

dbcon <- dbConnect(
  drv = MySQL(),
  host = "localhost",  # or your server address
  dbname = "iasamori",
  user = "iasamori",
  port = 3306
)

R[write to console]: Loading required package: ggthemes

R[write to console]: Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


R[write to console]: Loading required package: RSQLite

R[write to console]: Loading required package: ggpubr

R[write to console]: Loading required package: psych

R[write to console]: 
Attaching package: ‘psych’


R[write to console]: The following objects are masked from ‘package:ggplot2’:

    %+%, alpha


R[write to console]: Loading required package: lm.beta

R[write to console]: Loading required package: lattice

R[write to console]: 
Attaching package: ‘caret’


R[write to console]: The following object is masked from ‘package:purrr’:

    lift




In [21]:
%%R

comments_7 <- dbGetQuery(dbcon, "select * from `cmt7`")

one_gram_meta <- dbGetQuery(dbcon, "select * from `feat$meta_1gram$cmt7_en_dedup$message_id`")
one_gram_meta_wide <- importFeat(one_gram_meta)
one_gram_meta_wide <- one_gram_meta_wide[one_gram_meta_wide$"_total1grams" > 5, ]

message_ids <- one_gram_meta_wide$group_id

comments_7_cleaned <- comments_7[comments_7$message_id %in% message_ids, ] 

# dbWriteTable(db_con, "comments_7_cleaned", comments_7_cleaned,
#              overwrite=TRUE, row.names=FALSE, encoding="UTF-8

dbWriteTable(conn = dbcon,
             name = "cmt7_cln",
             value = comments_7_cleaned,
             overwrite = TRUE,
             append = FALSE,
             row.names = FALSE)

print(dim(comments_7))
print(dim(comments_7_cleaned))
head(comments_7_cleaned)

[1] 938852      8
[1] 569581      8
                                                                                                                    message
6                                    I can sympathize with you. I am so sorry for your loss. My son is presently in prison.
7                                                       Thank you for your compassion many blessings to you and your son 🤎
9                                                                                Another beautiful soul lost too soon. 💔🥺
10 You are so welcome, what a beautiful boy you had. The world lost out on this. But I am so happy you like it ❤️❤️❤️❤️🙌🙌🙏🙏
11                                                      This is absolutely beautiful tribute to my son … thank you 🥹😭💔🤍🕊️
13                                                    Illicit Fentanyl is destroying our country. Sorry for your loss. 💖🙏
       video_id create_time   message_id like_count parent_comment_id
6  7.257209e+18  1690164719 7.259

#### Indexing cleaned table

In [22]:
%%sql

CREATE INDEX idx_cmt7_cln_message ON cmt7_cln (message);
CREATE INDEX idx_cmt7_cln_video_id ON cmt7_cln (video_id);
CREATE INDEX idx_cmt7_cln_create_time ON cmt7_cln (create_time);
CREATE INDEX idx_cmt7_cln_message_id ON cmt7_cln (message_id);
CREATE INDEX idx_cmt7_cln_like_count ON cmt7_cln (like_count);
CREATE INDEX idx_cmt7_cln_parent_comment_id ON cmt7_cln (parent_comment_id);
CREATE INDEX idx_ccmt7_cln_reply_count ON cmt7_cln (reply_count);
CREATE INDEX idx_ccmt7_cln_month_yeart ON cmt7_cln (month_year);

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

++
||
++
++

### Making 1 gram of cleaned table

#### 1gram by message_id

In [23]:
!dlatkInterface.py --corpdb iasamori --corptable cmt7_cln --correl_field message_id \
 --add_ngrams -n 1 --feat_occ_filter --set_p_occ 0.0001




-----
DLATK Interface Initiated: 2025-06-06 04:25:55
-----
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_cln' AND column_name='message_id'
SQL QUERY: DROP TABLE IF EXISTS feat$1gram$cmt7_cln$message_id
SQL QUERY: CREATE TABLE feat$1gram$cmt7_cln$message_id ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id double, feat VARCHAR(36) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin, value INTEGER, group_norm DOUBLE, KEY `correl_field` (`group_id`), KEY `feature` (`feat`)) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_cln' AND column_name='message_id'
SQL QUERY: DROP TABLE IF EXISTS feat$meta_1gram$cmt7_cln$message_id
SQL QUERY: CREATE TABLE feat$meta_1gram$cmt7_cln$message_id ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id double, feat VARCHAR(16) CHARACTER SET utf8mb4 CO

#### 1gram by month

In [24]:
!dlatkInterface.py --corpdb iasamori --corptable cmt7_cln --correl_field month_year \
 --add_ngrams -n 1




-----
DLATK Interface Initiated: 2025-06-06 04:54:16
-----
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_cln' AND column_name='month_year'
SQL QUERY: DROP TABLE IF EXISTS feat$1gram$cmt7_cln$month_year
SQL QUERY: CREATE TABLE feat$1gram$cmt7_cln$month_year ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id text, feat VARCHAR(36) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin, value INTEGER, group_norm DOUBLE, KEY `correl_field` (`group_id`), KEY `feature` (`feat`)) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_cln' AND column_name='month_year'
SQL QUERY: DROP TABLE IF EXISTS feat$meta_1gram$cmt7_cln$month_year
SQL QUERY: CREATE TABLE feat$meta_1gram$cmt7_cln$month_year ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id text, feat VARCHAR(16) CHARACTER SET utf8mb4 COLLAT